<a href="https://colab.research.google.com/github/IvyWang845/Project-Practice-3-CNN-Image-Classification/blob/main/1_CNN_ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip show torch

In [ ]:
import torch # PyTorch
from torchvision import datasets # Datasets module
import torchvision.transforms as transforms # Image Transforms
from torch.utils.data.sampler import SubsetRandomSampler # Sampler

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,    # train=true => training set
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,    # train=false => test set
                                  download=True, transform=transform)

In [ ]:
# obtain training indices that will be used for validation

# 1. Create a list of indices of the training data
num_train = len(train_data)
print('num_train = len(train_data) ==> ', num_train)
indices = list(range(num_train))
print('len(indices) ==>', len(indices))

In [ ]:
# 2. Randomly Shuffle those indices
np.random.shuffle(indices)

In [ ]:
# 3. Slice the indices in 80-20 split
# percentage of training set to use as validation
valid_size = 0.2 # ie Train Set divided into two parts
                 # 80% Train 20% Validation
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

print('len(train_idx) ==> ', len(train_idx))
print('len(valid_idx) ==> ', len(valid_idx))

len(train_idx) ==>  48000
len(valid_idx) ==>  12000


In [ ]:
# define samplers for obtaining training and validation batches
# remember train_idx and valid_idx were the indices that we shuffled above
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare dataloaders
# number of subprocesses to use for data loading
num_workers = 0 # do not modify
# how many samples per batch to load
batch_size = 64 # ie 20 images per batch

# Training Set
train_loader = torch.utils.data.DataLoader(dataset=train_data, \
                                           batch_size=batch_size, \
                                           sampler=train_sampler, \
                                           num_workers=num_workers)
# Validation Set
valid_loader = torch.utils.data.DataLoader(dataset=train_data, \
                                           batch_size=batch_size, \
                                           sampler=valid_sampler, \
                                           num_workers=num_workers)
# Test Set
# Notice we have not used a 'sampler' here as it was not required
test_loader = torch.utils.data.DataLoader(dataset=test_data, \
                                          batch_size=batch_size, \
                                          num_workers=num_workers)

In [ ]:
import torch.nn as nn # nn module contains all the layers
import torch.nn.functional as F # same as nn, but a little different

In [ ]:
# Our CNN based neural architecture
# Let's build a simple one with only Convolutional, Linear
# and dropout layers
class MNISTModel1(nn.Module):
    # Here we define the neural architecture
    def __init__(self):
        super(MNISTModel1, self).__init__() # Initialize the nn module

        # Convolutional Layers
        # What shape/dimensions the first layer is going to see?
        # Do we need to have some padding for a kernel_size = 3?
        # Input Features = 1 x 28 x 28
        # Output Features = ???
        # Shape of a Convolutional Layer = (W - K + 2P)
        #                                  ------------ + 1
        #                                       S
        # where,
        #       W = Width/Height of previous layer = 28
        #       K = Filter Size = 3
        #       P = Padding = 0
        #       S = Stride = 1(default)
        # Therefore,
        #           if padding = 0
        #           Output Shape = ((28 - 3 + 2*0)/1)+1 = 26
        # We want the dimensions to stay the same so that there is no
        # loss of information when performing the convolution.
        # Hence,
        #       if padding = 1
        #       Output Shape = ((28 - 3 + 2*1)/1)+1 = 28
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, \
                               stride=1, padding=1) # Output Features = 8 x 28 x 28
        # Input Features = 8 x 28 x 28
        # Output Features = 16 x 28 x 28 | ((28 - 3 + 2*1)/1)+1 = 28
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, \
                               stride=1, padding=1)

        # Linear Layers
        # What shape the first linear layer is going see?
        # What are the total number of features given out by conv2?
        # Features = 16 x 28 x 28 = 12544
        # Therefore,
        self.linear1 = nn.Linear(in_features=12544, out_features=256)
        self.linear2 = nn.Linear(in_features=256, out_features=64)
        # Last linear layer should output 10 features as we are
        # Classifying the images in 10 categories
        self.linear3 = nn.Linear(in_features=64, out_features=10)

        # Dropout
        self.dropout = nn.Dropout(p=0.25)

    # Here we define the 'forward behaviour' of our neural architecture
    def forward(self, image_batch):
        # This is also the place where we add ACTIVATION functions
        image_batch = F.relu(input=self.conv1(image_batch))
        image_batch = F.relu(input=self.conv2(image_batch))

        # Remember that when passing image_batch through the Linear layers,
        # PyTorch expects:
        # >>> torch.Size([12, 256]) -> example values
            # 2d: [batch_size, num_features (aka: C * H * W)]
            # use for nn.Linear() input.
        # Therefore, we need to 'flatten' image_batch
        # image_batch = image_batch.view(batch_size, -1) --> batch size ???
        flat_image_batch = image_batch.view(image_batch.shape[0], -1)
        flat_image_batch = F.relu(input=self.linear1(flat_image_batch))
        # Let's add the dropout too
        flat_image_batch = self.dropout(F.relu(input=self.linear2(flat_image_batch)))
        # Final Layer of the network
        flat_image_batch = F.relu(input=self.linear3(flat_image_batch))
        # The output from the final layer is a tensor with 10 'logits'
        return flat_image_batch

In [ ]:
from torchsummary import summary

In [ ]:
# We can make the use of torchsummary library here to figure
# if we have done something wrong

# But first we need to tell PyTorch where to 'keep' the model
# On GPU or on CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('The model will run on', device)

# Initialize the model
mnist1 = MNISTModel1().to(device)
summary(model=mnist1, input_size=(1, 28, 28), batch_size=20) # Summarize

The model will run on cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [20, 8, 28, 28]              80
            Conv2d-2           [20, 16, 28, 28]           1,168
            Linear-3                  [20, 256]       3,211,520
            Linear-4                   [20, 64]          16,448
           Dropout-5                   [20, 64]               0
            Linear-6                   [20, 10]             650
Total params: 3,229,866
Trainable params: 3,229,866
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.06
Forward/backward pass size (MB): 2.93
Params size (MB): 12.32
Estimated Total Size (MB): 15.31
----------------------------------------------------------------


In [ ]:
!pip install poutyne

     |████████████████████████████████| 210 kB 12.8 MB/s 
     |████████████████████████████████| 408 kB 50.0 MB/s 


In [ ]:
from poutyne.framework import Model # The core datastructure of poutyne
                                    # https://poutyne.org/model.html

In [ ]:
from torch import optim # Optimizer: we need it to train our network

In [ ]:
# A pouytne training loop

# A few hyperparamters for the training loop
learning_rate = 0.1
epochs = 3 #train 3 go-through times

def poutyne_train(pytorch_model):

    # Select the optimizer and the loss function
    optimizer = optim.SGD(pytorch_model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    # Poutyne Model
    model = Model(pytorch_model, optimizer, loss_function, batch_metrics=['accuracy'])
    # Send the 'Poutyne model' on GPU/CPU whichever is available
    model.to(device)
    # Train
    model.fit_generator(train_loader, valid_loader, epochs=epochs)
    # Test
    test_loss, test_acc = model.evaluate_generator(test_loader)
    print(f'Test:\n\tLoss: {test_loss: .3f}\n\tAccuracy: {test_acc: .3f}')

    return None

In [ ]:
poutyne_train(mnist1)

Epoch: 1/3 Train steps: 750 Val steps: 188 14.60s loss: 0.509408 acc: 83.783333 val_loss: 0.155502 val_acc: 95.175000
Epoch: 2/3 Train steps: 750 Val steps: 188 14.20s loss: 0.142731 acc: 95.737500 val_loss: 0.106250 val_acc: 96.833333
Epoch: 3/3 Train steps: 750 Val steps: 188 14.28s loss: 0.093152 acc: 97.175000 val_loss: 0.102751 val_acc: 97.025000
Test steps: 157 1.80s test_loss: 0.088598 test_acc: 97.100000                                 
Test:
	Loss:  0.089
	Accuracy:  97.100
